# Data Engineering Zoomcamp 2024 - Homework 6
## Question 7: Most popular destination

In [1]:
import pyspark
from pyspark.sql import SparkSession

### Creating the PySpark consumer

In [2]:
pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"
kafka_clients_jar = 'org.apache.kafka:kafka-clients:3.4.1'

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("test_hw06_question7") \
    .config("spark.jars.packages", f"{kafka_jar_package},{kafka_clients_jar}") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/mhack/.pyenv/versions/3.10.3/envs/pyspark-kafka/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/mhack/.ivy2/cache
The jars for the packages stored in: /Users/mhack/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-62c992fd-5afa-47c0-840e-d0dea653c71f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found com.github.luben#zstd-jni;1.5.5-4 in central
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.0 in ce

In [4]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()


In [5]:
green_stream.printSchema()


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



## Question 6. Parsing the data

In [6]:
from pyspark.sql import types
from pyspark.sql import functions as F

In [7]:
schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.IntegerType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())


In [8]:
parsed_df = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [9]:
df_tripdata = parsed_df.withColumn("timestamp", F.current_timestamp())

In [10]:
df_popular_destinations = df_tripdata.groupBy( \
    F.window(F.col('timestamp'), '5 minutes'),
    F.col('DOLocationID')) \
    .count().orderBy(F.desc('count'))


In [ ]:
query = df_popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()
query.awaitTermination()


24/03/14 15:30:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/g0/707fw8gn4y1dd16yfrl8xpkm0000gn/T/temporary-afcd7ab5-5fcd-4cd6-a0c8-2b955fea0c73. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/14 15:30:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/14 15:30:11 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |8    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |8    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |7    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|82          |7    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |6    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|188         |5    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|239         |4    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|228         |4    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|189         |4    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|37          |3    |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|205         |3    |
|{2024-

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |1810 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |1449 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|41          |1336 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|75          |1276 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|166         |1205 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |974  |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |957  |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|236         |781  |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|95          |631  |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|223         |629  |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|238         |628  |
|{2024-

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |3108 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |2753 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|41          |2473 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|75          |2067 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |2065 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |2040 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|166         |1982 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|223         |1321 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|236         |1268 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|238         |1257 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|82          |1171 |
|{2024-

...

-------------------------------------------
Batch: 14
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |16779|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |14738|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|41          |13436|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|75          |11928|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |11379|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |10987|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|166         |10526|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|236         |7616 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|223         |7306 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|238         |7019 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|82          |6743 |
|{2024

-------------------------------------------
Batch: 15
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |17095|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |15071|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|41          |13609|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|75          |12204|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |11551|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |11169|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|166         |10605|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|236         |7717 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|223         |7383 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|238         |7120 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|82          |6893 |
|{2024

-------------------------------------------
Batch: 16
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |17365|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |15482|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|41          |13823|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|75          |12514|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |11718|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |11322|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|166         |10696|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|236         |7800 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|223         |7461 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|238         |7210 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|82          |7076 |
|{2024

-------------------------------------------
Batch: 17
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |17699|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |15898|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|41          |14031|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|75          |12795|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |11907|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |11498|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|166         |10821|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|236         |7902 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|223         |7532 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|238         |7301 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|82          |7264 |
|{2024

-------------------------------------------
Batch: 18
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|74          |17741|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|42          |15942|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|41          |14061|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|75          |12840|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|129         |11930|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|7           |11533|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|166         |10845|
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|236         |7913 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|223         |7542 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|238         |7318 |
|{2024-03-14 15:30:00, 2024-03-14 15:35:00}|82          |7292 |
|{2024